In [71]:
import pandas as pd
import numpy as np
import jieba
import jieba.analyse
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfTransformer,TfidfVectorizer
 
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold,StratifiedKFold
from scipy.sparse import csr_matrix, hstack
from sklearn.metrics import f1_score,accuracy_score,classification_report
import sklearn.metrics as metrics
from sklearn import svm
from sklearn.preprocessing import LabelEncoder
from collections import Counter
import pickle as pk
from sklearn import svm
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from fastFM import mcmc
from sklearn.naive_bayes import MultinomialNB

In [72]:
# list datalab file
!ls ./

hhh.ipynb               output.csv              test_public.csv
kkk.ipynb               stopWord.txt            train.csv
label_encoder.sav       submit.csv              人工智障.ipynb
multi_label_encoder.sav submit_example.csv


In [73]:
path ='./'

In [74]:
def get_data():
    train = pd.read_csv(path + 'train.csv')
    test = pd.read_csv(path + 'test_public.csv')
 
    train = train.sample(frac=1)
    train = train.reset_index(drop=True)
    
    data = pd.concat([train, test])
    
    lbl =  LabelEncoder()
#     lbl = MultiLabelBinarizer()
    lbl.fit(train['subject'])
    nb_classes = len(list(lbl.classes_))

    pk.dump(lbl, open('label_encoder.sav','wb'))
 
    subject = lbl.transform(train['subject'])
 
    y = []
    for i in list(train['sentiment_value']):
        y.append(i)
 
    y1= []
    for i in subject:
        y1.append(i)
 
    print(np.array(y).reshape(-1,1)[:,0])
    return data,train.shape[0],np.array(y).reshape(-1,1)[:,0],test['content_id'],np.array(y1).reshape(-1,1)[:,0]


In [75]:
def processing_data(data):
#     rule=re.compile("[^\u4e00-\u9fa5]")
#     data=re.sub(rule,"",data)
    word = jieba.cut(data)
    return ' '.join(word)

In [76]:
def pre_process():
  
    data,nrw_train,y,test_id,y1 = get_data()
 
    data['cut_comment'] = data['content'].map(processing_data)
    
    print('TfidfVectorizer')
    tf = TfidfVectorizer(ngram_range=(1,3),analyzer='char')
    discuss_tf = tf.fit_transform(data['cut_comment'])
#     print(discuss_tf)
 
    print('LSA')
    svd = TruncatedSVD(n_components=100, n_iter=10, random_state=42) 
    discuss_lsa = svd.fit_transform(discuss_tf)
    
    #print('LDA')
    #lda = LatentDirichletAllocation(n_components=10, random_state=2018)
    #discuss_lda = lda.fit_transform(discuss_tf)
    
    print('HashingVectorizer')
    ha = HashingVectorizer(ngram_range=(1,2),lowercase=False)
    discuss_ha = ha.fit_transform(data['cut_comment'])

    data = hstack((discuss_tf, discuss_ha)).tocsr()
 
    return data[:nrw_train],data[nrw_train:],y,test_id,y1

In [77]:
X,test,y,test_id,y1= pre_process()
N = 10
kf = StratifiedKFold(n_splits=N, random_state=2018).split(X,y)

/Users/xuqinqi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  


[ 1  0  1 ... -1  0  0]
TfidfVectorizer
LSA
HashingVectorizer


In [78]:
clf_list = []
from sklearn.linear_model import LogisticRegression
# clf = LogisticRegression(C=0.5)
# clf_list.append(clf)
# clf_list.append(clf)
# clf_list.append(clf)
# clf_list.append(clf)
# from sklearn.ensemble import RandomForestClassifier
# clf = RandomForestClassifier()
# clf_list.append(clf)
# clf_list.append(clf)
# clf_list.append(clf)
clf = svm.LinearSVC(loss='hinge', tol=1e-4, C=0.6)
clf_list.append(clf)
clf_list.append(clf)
clf_list.append(clf)
clf_list.append(clf)
#clf = MultinomialNB()
#clf = KNeighborsClassifier(n_neighbors=11)
clf_list.append(clf)
clf_list.append(clf)


In [79]:
y_train_oofp = np.zeros_like(y, dtype='float64')
y_train_oofp1 = np.zeros_like(y, dtype='float64')
 
y_test_oofp = np.zeros((test.shape[0], N))
y_test_oofp_1 = np.zeros((test.shape[0], N))

In [80]:
def micro_avg_f1(y_true, y_pred):
    return metrics.f1_score(y_true, y_pred, average='micro')

In [81]:
acc = 0
vcc = 0
for i ,(train_fold,test_fold) in enumerate(kf):
    X_train, X_validate, label_train, label_validate,  label_1_train, label_1_validate,= X[train_fold, :], X[test_fold, :], y[train_fold], y[test_fold], y1[train_fold], y1[test_fold]
#     print(X_train)
    clf[0].fit(X_train, label_train)
    
    val_ = clf.predict(X_validate)
    y_train_oofp[test_fold] = val_
#     print(val_)
#     print('sentiment_value_f1:%f' % micro_avg_f1(label_validate, val_))
    print('sentiment_value_f1:%f' % micro_avg_f1(label_validate, val_))
    acc += micro_avg_f1(label_validate, val_)
    result = clf.predict(test)
    y_test_oofp[:, i] = result
 
    clf[1].fit(X_train, label_1_train)
    val_1 = clf.predict(X_validate)
    y_train_oofp1[test_fold] = val_
 
    print('subject:%f' % micro_avg_f1(label_1_validate, val_1))
    vcc += micro_avg_f1(label_1_validate, val_1)
    result = clf.predict(test)
    y_test_oofp_1[:, i] = result
 
print(acc/N)
print(vcc/N)
# 8
# 0.7174025487306597
# 0.682218975900569
# ngram = 3 
# 0.7314751283426579
# 0.663719536650388

sentiment_value_f1:0.730924
subject:0.666667
sentiment_value_f1:0.747739
subject:0.671357
sentiment_value_f1:0.733668
subject:0.655276
sentiment_value_f1:0.736683
subject:0.649246
sentiment_value_f1:0.718593
subject:0.658291
sentiment_value_f1:0.738693
subject:0.660302
sentiment_value_f1:0.749497
subject:0.654930
sentiment_value_f1:0.720322
subject:0.662978
sentiment_value_f1:0.723340
subject:0.656942
sentiment_value_f1:0.715292
subject:0.701207
0.7314751283426579
0.663719536650388


In [85]:
test_id_list = []
test_id_num = []
lbl = pk.load(open('label_encoder.sav','rb'))
res_2 = []
for i in range(y_test_oofp_1.shape[0]):
    tmp = []
    for j in range(N):
        tmp.append(int(y_test_oofp_1[i][j]))
    word_counts = Counter(tmp)
    yes = word_counts.most_common()
    res_2.append(lbl.inverse_transform(yes[0][0]))
    test_id_list.append(test_id[i])
#     print(yes)
#     num = 0 
#     for j in yes:
#         if(num==0 or j[1] > 3):
# #             print(lbl.inverse_transform(j[0]))
#             res_2.append(lbl.inverse_transform(j[0]))
#             test_id_list.append(test_id[i])
#             num += 1
#     test_id_num.append(num)
# print(list(res_2))

/Users/xuqinqi/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/xuqinqi/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/xuqinqi/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/xuqinqi/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty ar

/Users/xuqinqi/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/xuqinqi/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/xuqinqi/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/xuqinqi/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty ar

/Users/xuqinqi/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/xuqinqi/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/xuqinqi/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/xuqinqi/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty ar

In [86]:
res = []
for i in range(y_test_oofp.shape[0]):
    tmp = []
    for j in range(N):
        tmp.append(y_test_oofp[i][j])
#     word_counts = Counter(tmp)
#     yes = word_counts.most_common()
#     print(yes, test_id_num[i])
#     for j in range(test_id_num[i]):
#         res.append(yes[j][0])
    res.append(max(set(tmp), key=tmp.count))

In [87]:
print(len(res))
print(len(res_2))
print(len(test_id_list))

# result = pd.read_csv(path + 'output.csv')
result = pd.DataFrame()
result['content_id'] = list(test_id_list)
 
result['subject'] = list(res_2)
result['subject'] = result['subject']
 
result['sentiment_value'] = list(res)
result['sentiment_value'] = result['sentiment_value'].astype(int)
 
result['sentiment_word'] = ''
result.to_csv('submit.csv',index=False)

2364
2364
2364


In [88]:
len(result)
result

,content_id,subject,sentiment_value,sentiment_word
0,XuPwKCnA2fqNh5vm,价格,1,
1,2jNbDn85goX3IuPE,价格,0,
2,hLgEADQ8sUnvGFK9,价格,0,
3,nZmM7LQsfr03wUaz,价格,0,
4,pwd8MnrthDqLZafe,价格,0,
5,cudSKGTyMLbIt8k3,价格,0,
6,U5RMQtopwFzDVxHm,价格,0,
7,Kh0ayJjuTrzwPAs1,价格,0,
8,YZovRtEAyQJgTi1I,价格,0,
9,BeSIAUzoTWH9muYl,配置,1,
